In [37]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

In [38]:
df = pd.read_csv("train.csv")
df = df.rename(columns={"Class": "label"})


In [40]:
df.head()
# #df['age']
# count = 0
# for column in df['age']:
#     count +=1
#     if column == '?':
#         print(count)
#         print("question")

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,label
0,1,1,1,1,0,0,1,1,0,0,...,f,White-European,no,no,'United States',no,6,'18 and more',Self,NO
1,1,1,0,1,0,0,0,1,0,1,...,m,Latino,no,yes,Brazil,no,5,'18 and more',Self,NO
2,1,1,0,1,1,0,1,1,1,1,...,m,Latino,yes,yes,Spain,no,8,'18 and more',Parent,YES
3,1,1,0,1,0,0,1,1,0,1,...,f,White-European,no,yes,'United States',no,6,'18 and more',Self,NO
4,1,0,0,0,0,0,0,1,0,0,...,f,?,no,no,Egypt,no,2,'18 and more',?,NO


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 21 columns):
A1                 598 non-null int64
A2                 598 non-null int64
A3                 598 non-null int64
A4                 598 non-null int64
A5                 598 non-null int64
A6                 598 non-null int64
A7                 598 non-null int64
A8                 598 non-null int64
A9                 598 non-null int64
A10                598 non-null int64
age                598 non-null int64
gender             598 non-null object
ethnicity          598 non-null object
jundice            598 non-null object
austim             598 non-null object
contry_of_res      598 non-null object
used_app_before    598 non-null object
result             598 non-null int64
age_desc           598 non-null object
relation           598 non-null object
label              598 non-null object
dtypes: int64(12), object(9)
memory usage: 98.2+ KB


In [42]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [43]:
random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)

In [44]:
data = train_df.values
data[:5]

array([[1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 26, 'f', 'White-European', 'no',
        'no', "'United States'", 'no', 6, "'18 and more'", 'Self', 'NO'],
       [1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 24, 'm', 'Latino', 'no', 'yes',
        'Brazil', 'no', 5, "'18 and more'", 'Self', 'NO'],
       [1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 27, 'm', 'Latino', 'yes', 'yes',
        'Spain', 'no', 8, "'18 and more'", 'Parent', 'YES'],
       [1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 35, 'f', 'White-European', 'no',
        'yes', "'United States'", 'no', 6, "'18 and more'", 'Self', 'NO'],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 40, 'f', '?', 'no', 'no', 'Egypt',
        'no', 2, "'18 and more'", '?', 'NO']], dtype=object)

In [45]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [46]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [47]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        type_of_feature = FEATURE_TYPES[column_index]
        if type_of_feature == "continuous":
            potential_splits[column_index] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[column_index].append(potential_split)
        
        # feature is categorical         
        else:
            potential_splits[column_index] = unique_values
    
    return potential_splits

In [48]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [49]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [50]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [51]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value


In [52]:
#check_purpose
for column in df.columns:
    print(column,"-",df[column].unique())

A1 - [1 0]
A2 - [1 0]
A3 - [1 0]
A4 - [1 0]
A5 - [0 1]
A6 - [0 1]
A7 - [1 0]
A8 - [1 0]
A9 - [0 1]
A10 - [0 1]
age - [ 26  24  27  35  40  36  17  64  29  33  18  31  30  34  38  42  43  48
  37  55  50  53  20  28  21 383  47  32  44  19  58  45  22  39  25  23
  54  60  41  46  56  61  59  52  49  51]
gender - ['f' 'm']
ethnicity - ['White-European' 'Latino' '?' 'Others' 'Black' 'Asian'
 "'Middle Eastern '" 'Pasifika' "'South Asian'" 'Hispanic' 'Turkish']
jundice - ['no' 'yes']
austim - ['no' 'yes']
contry_of_res - ["'United States'" 'Brazil' 'Spain' 'Egypt' "'New Zealand'" 'Bahamas'
 'Burundi' 'Austria' 'Argentina' 'Jordan' 'Ireland'
 "'United Arab Emirates'" 'Afghanistan' 'Lebanon' "'United Kingdom'"
 "'South Africa'" 'Italy' 'Pakistan' 'Bangladesh' 'Chile' 'France' 'China'
 'Australia' 'Canada' "'Saudi Arabia'" 'Netherlands' 'Romania' 'Sweden'
 'Tonga' 'Oman' 'India' 'Philippines' "'Sri Lanka'" "'Sierra Leone'"
 'Ethiopia' "'Viet Nam'" 'Iran' "'Costa Rica'" 'Germany' 'Mexico' 'Rus

In [62]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15 #if cross then continuous value
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]
            
            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [63]:
#check purpose

feature_types = determine_type_of_feature(df)
i=0
for column in df.columns:
    print(column,"-",feature_types[i])
    i += 1 


A1 - categorical
A2 - categorical
A3 - categorical
A4 - categorical
A5 - categorical
A6 - categorical
A7 - categorical
A8 - categorical
A9 - categorical
A10 - categorical
age - continuous
gender - categorical
ethnicity - categorical
jundice - categorical
austim - categorical
contry_of_res - categorical
used_app_before - categorical
result - categorical
age_desc - categorical
relation - categorical


IndexError: list index out of range

In [55]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        sub_tree = {question: []}
        
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [75]:


tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree,width=50)



{'A9 = 1': [{'A1 = 1': [{'result = 6': ['NO',
                                        'YES']},
                        {'result = 9': ['YES',
                                        'NO']}]},
            {'result = 7': ['YES',
                            {'result = 8': ['YES',
                                            'NO']}]}]}


In [57]:
example = test_df.iloc[0]
example

A1                                      1
A2                                      0
A3                                      0
A4                                      0
A5                                      1
A6                                      1
A7                                      0
A8                                      1
A9                                      0
A10                                     1
age                                    22
gender                                  f
ethnicity                           Black
jundice                                no
austim                                 no
contry_of_res      'United Arab Emirates'
used_app_before                        no
result                                  5
age_desc                    '18 and more'
relation                             Self
label                                  NO
Name: 394, dtype: object

In [58]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [59]:
classify_example(example, tree)

'NO'

In [60]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, args=(tree,), axis=1)
    df["classification_correct"] = df["classification"] == df["label"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [61]:
accuracy = calculate_accuracy(test_df, tree)
accuracy

0.9